In [2]:
import pandas as pd

df = pd.read_csv('train.csv')

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')


In [3]:
from nltk.stem import WordNetLemmatizer, PorterStemmer

def preprocesar(c):

    wnl = WordNetLemmatizer()
    ps = PorterStemmer()
    for r in c:
        s = r
        s = s.lower()
        print(s)
    

ModuleNotFoundError: No module named 'nltk'

In [ ]:
df['title'] = preprocesar(df['title'])